In [1]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip \
    -O /tmp/horse-or-human.zip
    
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip \
    -O /tmp/validation-horse-or-human.zip

--2020-07-04 22:40:38--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.24.16, 172.217.27.144, 172.217.160.112, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.24.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149574867 (143M) [application/zip]
Saving to: ‘/tmp/horse-or-human.zip’

/tmp/horse-or-human 100%[===================>] 142.65M  4.05MB/s    in 38s     

2020-07-04 22:41:18 (3.72 MB/s) - ‘/tmp/horse-or-human.zip’ saved [149574867/149574867]

--2020-07-04 22:41:18--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.27.144, 172.217.160.112, 216.58.200.240, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.27.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11

In [3]:
import os 
import zipfile

local_zip = '/tmp/horse-or-human.zip'
zip_ref= zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/horse-or-human')
local_zip = '/tmp/validation-horse-or-human.zip'
zip_ref= zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/validation-horse-or-human')
zip_ref.close()

In [6]:
train_horse_dir = os.path.join('/tmp/horse-or-human/horses')
train_human_dir = os.path.join('/tmp/horse-or-human/humans')
validation_horse_dir = os.path.join('/tmp/validation-horse-or-human/validation-horses')
validation_human_dir = os.path.join('/tmp/validation-horse-or-human/validation-humans')

In [7]:
import tensorflow as tf
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16,(3,3), activation = 'relu', input_shape = (150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32,(3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64,(3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64,(3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 64)        3

In [10]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss = 'binary_crossentropy',
              optimizer = RMSprop(lr=0.001),
              metrics = ['acc'])

In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1/255)
validation_datagen = ImageDataGenerator(rescale = 1/255)

train_generator = train_datagen.flow_from_directory(
        '/tmp/horse-or-human',
        target_size = (150,150),
        batch_size = 128,
        class_mode = 'binary')

validation_generator = validation_datagen.flow_from_directory(
        '/tmp/validation-horse-or-human',
        target_size = (150,150),
        batch_size = 32,
        class_mode = 'binary')

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


In [14]:
history = model.fit_generator(
        train_generator,
        steps_per_epoch = 8,
        epochs = 15,
        verbose = 1,
        validation_data = validation_generator,
        validation_steps = 8)

Epoch 1/15
8/8 [==============================] - 56s 7s/step - loss: 0.7285 - acc: 0.6618 - val_loss: 3.6810 - val_acc: 0.5000
Epoch 2/15
8/8 [==============================] - 52s 6s/step - loss: 0.9410 - acc: 0.6819 - val_loss: 0.3226 - val_acc: 0.8711
Epoch 3/15
8/8 [==============================] - 62s 8s/step - loss: 0.3426 - acc: 0.8432 - val_loss: 1.7034 - val_acc: 0.7891
Epoch 4/15
8/8 [==============================] - 52s 6s/step - loss: 0.2307 - acc: 0.9143 - val_loss: 1.1905 - val_acc: 0.8242
Epoch 5/15
8/8 [==============================] - 66s 8s/step - loss: 0.1513 - acc: 0.9404 - val_loss: 0.8714 - val_acc: 0.7969
Epoch 6/15
8/8 [==============================] - 61s 8s/step - loss: 0.2492 - acc: 0.9132 - val_loss: 0.6827 - val_acc: 0.8672
Epoch 7/15
8/8 [==============================] - 59s 7s/step - loss: 0.6615 - acc: 0.8053 - val_loss: 0.7156 - val_acc: 0.8008
Epoch 8/15
8/8 [==============================] - 61s 8s/step - loss: 0.1423 - acc: 0.9477 - val_loss: 1

In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(150, 150))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " is a human")
  else:
    print(fn + " is a horse")
 